# SmartFigure: interactive tests for parameter-change hooks

This notebook tests the **new parameter-change hook API** you added to `SmartFigure`.

What “interactive” means here
- You will **move sliders** in the UI and **watch the plot + logs update live**.
- No simulated events are used.

Before you start
- Make sure you are running in a widget-capable environment (JupyterLab / Notebook).
- If widgets don’t render at all, you likely need `ipywidgets` enabled for your Jupyter install.
  - JupyterLab 3+ typically works out of the box once `ipywidgets` is installed.
  - Plotly `FigureWidget` also depends on widget support.

**How to use this notebook**
1. Run cells top-to-bottom.
2. After you see the figure + slider(s), follow the instruction blocks that say “➡️ Do this…”.
3. If your hooks work, you will see log lines and assertions will pass.


## 1) Imports and robust loading

This cell tries a couple of import paths.

If it fails, edit the import section to match your project layout.

➡️ Do this: run the cell and confirm it prints the class location and version-ish info.


In [8]:
from pathlib import Path
import sys
ROOT = Path.cwd().resolve().parents[1]  
sys.path.insert(0, str(ROOT))
from gu_toolkit.SmartFigure import SmartFigure

In [9]:
import sympy as sp

## 2) Create a figure with a real parameter slider

We create a simple plot with a parameter `a` so that `SmartFigure` creates a slider.

➡️ Do this:
- Run the cell.
- You should see a plot and a parameter slider for `a`.
- Move the slider and confirm the curve changes (even before hooks).


In [10]:

x, a = sp.symbols("x a")

fig = SmartFigure(x_range=(-6, 6), y_range=(-3, 3))
fig.title = r"Hook tests: $y = a \sin(x)$"

# This should create the slider for parameter 'a'
fig.plot(x, a * sp.sin(x), id="a_sin")

display(fig)

# Convenience: try to reduce spam if SmartFloatSlider exposes inner slider
try:
    # Many implementations store the underlying FloatSlider at `.slider`
    fig._params[a].slider.continuous_update = False
    print("Set continuous_update=False to make hook logs easier to read.")
except Exception:
    print("Could not set continuous_update; that's fine.")

# Sanity check: your new API should exist
if not hasattr(fig, "add_param_change_hook"):
    raise AttributeError(
        "SmartFigure is missing add_param_change_hook. "
        "Make sure you've applied the code changes, then restart kernel and re-run."
    )


OneShotOutput()

Set continuous_update=False to make hook logs easier to read.


## 3) Set up a live log panel

We’ll capture hook activity in an `Output` widget to keep the notebook tidy.

➡️ Do this:
- Run the cell.
- You should see an empty log panel.


In [12]:
import ipywidgets as widgets
log = widgets.Output(
    layout=widgets.Layout(
        border="1px solid #ddd",
        padding="8px",
        height="180px",
        overflow="auto",
    )
)
display(widgets.HTML("<b>Hook log</b>"))
display(log)

def log_line(msg: str) -> None:
    with log:
        print(msg)


HTML(value='<b>Hook log</b>')

Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…

## 4) Test A: Autogenerated ids (`"hook:1"`, `"hook:2"`, …) and basic hook firing

We register one hook with no `hook_id`. It should return `"hook:1"` (assuming no earlier hooks).

➡️ Do this:
1. Run the cell.
2. Confirm it prints the returned hook id.
3. Move slider `a` a few times.
4. You should see log lines like `HOOK hook:1: old -> new`.


In [13]:
log.clear_output()

def hook_basic(change: dict) -> None:
    # The change dict is traitlets/ipywidgets standard:
    old = change.get("old", None)
    new = change.get("new", None)
    log_line(f"HOOK hook:1 (basic): {old} -> {new}")

hid1 = fig.add_param_change_hook(hook_basic)
print("Returned hook id:", hid1)

assert hid1 == "hook:1", f"Expected 'hook:1', got {hid1!r}"

log_line("➡️ Now move slider a. You should see HOOK hook:1 lines.")


Returned hook id: hook:1


## 5) Test B: Replace an existing hook (same `hook_id`)

If you call `add_param_change_hook(..., hook_id=hid1)` again, the existing hook should be replaced.

➡️ Do this:
1. Run the cell.
2. Move slider `a`.
3. You should **only** see the `REPLACED` message, not the old `basic` one.


In [14]:
log.clear_output()

def hook_replaced(change: dict) -> None:
    old = change.get("old", None)
    new = change.get("new", None)
    log_line(f"HOOK {hid1} (REPLACED): {old} -> {new}")

hid1b = fig.add_param_change_hook(hook_replaced, hook_id=hid1)
print("Returned hook id:", hid1b)

assert hid1b == hid1, "Replacing should return the same id."

log_line("➡️ Now move slider a. You should see only the REPLACED lines.")


Returned hook id: hook:1


## 6) Test C: User supplies `"hook:N"` and the internal counter bumps

Rule: if the user provides a hook id like `"hook:10"`, then the internal counter must be bumped
so later auto ids will not collide.

➡️ Do this:
1. Run the cell.
2. Confirm that the next autogenerated id becomes `"hook:11"`.


In [15]:
log.clear_output()

# Add a no-op hook with an explicit id in the auto-id namespace
hid10 = fig.add_param_change_hook(lambda ch: None, hook_id="hook:10")
print("Explicit hook id:", hid10)

# Next auto id should be hook:11 (or higher if you already created higher hook ids)
hid_next = fig.add_param_change_hook(lambda ch: None)
print("Next autogenerated id:", hid_next)

assert isinstance(hid_next, str) and hid_next.startswith("hook:"), "Auto hook id should be a string 'hook:N'."
n = int(hid_next.split(":")[1])
assert n >= 11, f"Expected counter bump to at least 11, got {hid_next!r}"

log_line("✅ Counter bump check passed.")


Explicit hook id: hook:10
Next autogenerated id: hook:11


## 7) Test D: Non-string, hashable hook ids work (e.g., tuples)

The new requirement: `hook_id` can be any **hashable** key (tuple, int, Enum, etc.).

➡️ Do this:
1. Run the cell.
2. Move slider `a`.
3. You should see a line for the tuple-id hook.


In [16]:
log.clear_output()

tuple_id = ("analysis", 1)

def hook_tuple_id(change: dict) -> None:
    log_line(f"HOOK {tuple_id!r}: new={change.get('new', None)}")

returned = fig.add_param_change_hook(hook_tuple_id, hook_id=tuple_id)
print("Returned hook id:", returned)

assert returned == tuple_id

log_line("➡️ Now move slider a. You should see HOOK ('analysis', 1) lines.")


Returned hook id: ('analysis', 1)


## 8) Test E: Unhashable hook ids raise a clear error

A dict key must be hashable. This cell checks that passing an unhashable id fails.

➡️ Do this:
- Run the cell.
- You should see a message confirming a `TypeError` (or similar) was raised.


In [17]:
log.clear_output()

try:
    fig.add_param_change_hook(lambda ch: None, hook_id=[])  # list is unhashable
    raise AssertionError("Expected an error for unhashable hook_id, but none was raised.")
except TypeError as e:
    print("✅ Caught expected TypeError:", e)


✅ Caught expected TypeError: hook_id must be hashable (usable as a dict key), got <class 'list'>


## 9) Test F: Hook exceptions do not break the figure (warnings only)

Your implementation should catch hook exceptions so that:
- the plot still updates
- other hooks still run
- the notebook stays interactive

➡️ Do this:
1. Run the cell.
2. Move slider `a`.
3. You should see:
   - the normal log hook line
   - and a warning about the failing hook
4. The plot should still update.


In [18]:
log.clear_output()

def hook_ok(change: dict) -> None:
    log_line(f"HOOK ok: new={change.get('new', None)}")

def hook_fail(change: dict) -> None:
    raise RuntimeError("Intentional test failure from hook_fail")

fig.add_param_change_hook(hook_ok, hook_id="ok-hook")
fig.add_param_change_hook(hook_fail, hook_id="fail-hook")

log_line("➡️ Now move slider a. Expect: OK log line + a warning, and plot still updates.")


## 10) Optional: Visual confirmation checklist

Use this as a quick checklist while you drag the slider.

- ✅ The curve visibly changes when you change `a`
- ✅ The hook log updates after slider moves
- ✅ Replacing a hook changes what is logged
- ✅ Providing `"hook:10"` causes the next auto id to be `"hook:11"` (or higher)
- ✅ Tuple ids work
- ✅ Unhashable ids raise a clear error
- ✅ A failing hook emits a warning but does not break the plot or other hooks
